In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

from functions import import_imagedata, label_oh_tf, ImageProcessor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, models, transforms
import torch.optim as optim
from torch.optim import lr_scheduler

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from torchvision.models import vgg16
from torchvision.models import resnet101

import cv2

from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

import random
import pickle

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"

In [3]:
model_resnet = resnet101(weights="IMAGENET1K_V1")
print(model_resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
# Step 1: Initialize model with the best available weights
#model_resnet_featlayers = resnet101(weights="IMAGENET1K_V1").features#.to(device)
#model_resnet_featlayers.eval()
newmodel = torch.nn.Sequential(*(list(model_resnet.children())[:-1]))
newmodel=newmodel.to(device)
print(newmodel)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [5]:
# Step 2: Initialize the inference transforms
# This does some preprocessing behind the scenes,

# 1) Resized of the input to resize_size=[256];
# 2) Followed by a central crop of crop_size=[224];

# vgg16 and resnet accept input image size of 224×224



#print(preprocess)

In [6]:
# create MLP, linear classification, function

class Three_Lay_MLP(nn.Module):
    def __init__(self):
        super(Three_Lay_MLP, self).__init__()
        
        self.lins = nn.Sequential(
            nn.Linear(2048, 100), #1024x7 and 1024x100. 7, 1024
            nn.ReLU(),
            nn.Linear(100,100),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(100,11),
            nn.Linear(11,11),
            nn.Softmax(),
        )
    def forward(self, x):
        x = self.lins(x)
        return x
        
#lin    conv output -> 100
#relu    ()
#lin    100 -> 100
#relu    ()
#do     ~0.5
#lin    100 -> 11
#softmax ()

In [7]:
def metrics(label, prediction): #TypeError: Singleton array tensor(3) cannot be considered a valid collection.
    #print('l  ',label) #len(11)
    label= np.array(label)
    #print(type(label)) #'y_pred' parameter of f1_score must be an array-like or a sparse matrix. Got 7 instead.
    #label = label.argmax()
    #print('l  ',label)
    predictions_np = prediction.detach().numpy()
    #print('p  ',predictions_np)
    predicted_classes = np.argmax(predictions_np, axis=0)
    #predicted_classes = prediction.argmax()
    avg_f1_score = f1_score(label, predicted_classes, average='macro')
    acc = accuracy_score(label, predicted_classes)
    
    return avg_f1_score, acc

In [8]:
"""from sklearn.preprocessing import StandardScaler

# creating a function for normalizing the feature vector
# using a standard scaling (results in mean=0 and standard deviation=1)
def scale_feats(feat_vec):
  # Scaling the features to the same range of values
  scaler = StandardScaler()
  scaler.fit(feat_vec)
  scaled_feat_vec = scaler.transform(feat_vec)
  print("\n A peek at the scaled dataset features: \n"+str(scaled_feat_vec))

  return scaled_feat_vec

# normalize the feature vector
scaled_feats_28 = scale_feats(feats_28)"""

'from sklearn.preprocessing import StandardScaler\n\n# creating a function for normalizing the feature vector\n# using a standard scaling (results in mean=0 and standard deviation=1)\ndef scale_feats(feat_vec):\n  # Scaling the features to the same range of values\n  scaler = StandardScaler()\n  scaler.fit(feat_vec)\n  scaled_feat_vec = scaler.transform(feat_vec)\n  print("\n A peek at the scaled dataset features: \n"+str(scaled_feat_vec))\n\n  return scaled_feat_vec\n\n# normalize the feature vector\nscaled_feats_28 = scale_feats(feats_28)'

In [9]:
# Temi code. week 6. notebook used vgg16 pretrained features, and trained a MLP for classification of beans
# this is the method I am following 

IP = ImageProcessor(device='cpu')

# Temi func. get features from passing through images into vgg16 conv/ feature layers.
# creating a function to get features learnt in the pretrained VGG16
# extracting these features for our own list of images
def get_img_feats(img_path): 
    #print(img_path)
    img = cv2.imread(img_path) #
    #print(img)
    img = IP.blank_padding(img, (224,224))
    img = IP.to_tensor(img).to(device)
    #print('image shape post read and pad: \n ',img.shape)
    # Step 3: Apply inference preprocessing transforms
    #img = preprocess(img).unsqueeze(0) # preprocess is the model with weights
    #img = preprocess(img)#.unsqueeze(0)

    # Step 4: Use the model and print the predicted category
    #print('get_img_feats img', img.shape) 
    #auto_feats = auto_feats.detach().numpy() 
    #Tensor.cpu() to copy the tensor to host memory first
    auto_feats = newmodel(img).squeeze(0)
    
    
    #print('get_img_feats auto_feats', auto_feats.shape)
    
    # AttributeError: 'builtin_function_or_method' object has no attribute 'detach' --- current error
    #AttributeError: 'numpy.ndarray' object has no attribute 'to'
    auto_feats = auto_feats.cpu().detach().numpy()
    
    #print('get_img_feats auto_feats detached', auto_feats.shape)
    auto_feats = np.mean(auto_feats, axis=1,keepdims=False) #, keepdims=False
    #print('get_img_feats auto_feats np mean', auto_feats.shape)
    #print('get_img_feats autofeats', auto_feats)
    #print('auto_feat: \n ', auto_feats)
    auto_feats= torch.tensor(auto_feats, dtype=torch.float32)
    return auto_feats.to(device)


# class to manage data and turn imgs into vgg16 features

"""class AutoFeature(): # file_path, number
    def __init__(self, img_files, labels):
        self.img_files = img_files
        self.label = label_oh_tf(labels, device='cpu', num_classes=11)
        
        #self.labels=[label_oh_tf(label, device='cpu', num_classes=11) for label in labels]
        #self.labels= [int(label) for label in labels]
        #self.labels = torch.tensor(labels, dtype=torch.long) # tensofy the labels
        #self.labels = label_oh_tf(labels, device, 11)
    #def __len__(self):
    #    return len(self.labels)
    def __getitem__(self):
        #given an index, will return items within that index range
        # extracting features using pretrained model
        feats = get_img_feats(self.img_files) #[idx]
        
        feats_tensor = torch.tensor(feats, dtype=torch.float32)
        #print(type(self.labels))
        #print(self.labels[3])
        return feats_tensor, self.label #[idx]"""

"class AutoFeature(): # file_path, number\n    def __init__(self, img_files, labels):\n        self.img_files = img_files\n        self.label = label_oh_tf(labels, device='cpu', num_classes=11)\n        \n        #self.labels=[label_oh_tf(label, device='cpu', num_classes=11) for label in labels]\n        #self.labels= [int(label) for label in labels]\n        #self.labels = torch.tensor(labels, dtype=torch.long) # tensofy the labels\n        #self.labels = label_oh_tf(labels, device, 11)\n    #def __len__(self):\n    #    return len(self.labels)\n    def __getitem__(self):\n        #given an index, will return items within that index range\n        # extracting features using pretrained model\n        feats = get_img_feats(self.img_files) #[idx]\n        \n        feats_tensor = torch.tensor(feats, dtype=torch.float32)\n        #print(type(self.labels))\n        #print(self.labels[3])\n        return feats_tensor, self.label #[idx]"

In [10]:
# get len of 

file_path = r'/its/home/nn268/optics/AugmentedDS_IDSW/'
random_seed =1
img_len = len(os.listdir(file_path))

ids = np.arange(0, img_len)
#print(ids[4])


train_ids, test_ids = train_test_split(ids, test_size=0.2, train_size=0.8,
                                 random_state=random_seed, shuffle=True)
train_ids, val_ids = train_test_split(train_ids, test_size=0.1, train_size=0.8,
                                 random_state=random_seed, shuffle=True)


In [11]:
wandb.login()


wandb: Currently logged in as: naughticalnonsence (antvis). Use `wandb login --relogin` to force relogin


True

In [12]:

title = f'IDSW_RESNET101_MLP_112023'
save_dict = {'Run' : title,
            'Current_Epoch': 0}

save_location = r'pickles/'

In [13]:
from torch.utils.data import DataLoader
import wandb
from fns4wandb import train_log, build_optimizer
from copy import deepcopy
from tqdm import tqdm

# split data intro train, val, test

# instancing the autofeature class
# creating an object for train, val test

#train_set = AutoFeature(x[train_ids], y[train_ids])
#val_set = AutoFeature(x[val_ids], y[val_ids])
#test_set = AutoFeature(x[test_ids], y[test_ids])

#print('train set 0: \n',train_set[0])
#print('what is train set?', type(train_set))
#print('train set len', len(train_set))



#x_test, y_test = test_set
#x_test = [i[0] for i in test_set]
#y_test = [i[1] for i in test_set]

#train_dl = DataLoader(train_set, batch_size=1)
#val_dl = DataLoader(val_set, batch_size=1)
#test_dl = DataLoader(test_set, batch_size=1)




config = dict(
    epochs= 40, #30, 
    learning_rate =1e-5,
    architecture ='CNN',
    optimizer= 'adam',
    weight_decay= 4e-5,
    ks = 3,
    scheduler=0.2,
    f_lin_lay = 7168 #1024*7 = 7168
)

# pass in ids
# create class instance for single ids
# index that class to get img feature and label FOR THAT ids.
# per epoch



def train_model(model, loss_fn, train_ids, val_ids, config, best_acc=0): #train_dl, val_dl, 
    #wandb.watch(model, loss_fn, log='all', log_freq=10)
    
    lr = config['learning_rate'] #1e-5 #config.learning_rate
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)#build_optimizer(model, optimizer=torch.optim.Adam(model.parameters(), lr=lr))#config.optimizer, config.learning_rate, config.weight_decay)
    scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=config['scheduler'], last_epoch=-1) #gamma=config.scheduler, last_epoch=-1)
    
    x, y = import_imagedata(file_path)
    #print(x[1].shape)
    
    model.train()
    
    t_loss_list = []
    v_loss_list = []
    t_predict_list = []
    v_predict_list = []
    t_accuracy_list = []
    v_accuracy_list = []
    t_label_list = []
    v_label_list = []
    #labels = []
    t_correct = 0
    v_correct = 0
    total_epochs = 0
    
    for epoch in tqdm(range(config['epochs'])): #config.epochs)):
        print('Epoch:   ', epoch)
        if epoch == 0:
            best_model = deepcopy(model)
        #train_ids = random.shuffle(train_ids)
        #print(type(train_ids))
        print('training...')
        for idx in train_ids: #batch,(x_train, y_train) in enumerate(train_dl)
            
            #print('idx    ',idx)
            #print('train x[idx]',x[idx].shape)
            #print('x:   ',x[idx])
            x_train = get_img_feats(x[idx])
            #print('x_train:    ',x_train)
            #print('train, x_train.shape ', x_train.shape)
            tensor = torch.tensor(x_train, dtype=torch.float32)
            tensor = tensor.flatten()

            train_prediction = model.forward(tensor)
            #print(prediction.shape)
            #print(train_prediction.argmax())
            train_label = label_oh_tf(y[idx], device, num_classes=11)
            #print(train_label.argmax())
            t_loss = loss_fn(train_prediction, train_label)
            
            #train_avg_f1_score, train_acc = metrics(train_label, train_prediction)
            if train_prediction.argmax() == train_label.argmax():
                t_correct+=1
                #print ('Train accuracy:    ',t_correct/len(train_ids))
            t_loss_list.append(t_loss)
            t_predict_list.append(train_prediction)
            train_acc = t_correct/len(train_ids)
            t_label_list.append(train_label)
            
            optimizer.zero_grad()
            t_loss.backward()
            
            optimizer.step()
            scheduler.step()
            
        print('validation... ')
        for idx in val_ids:
            
            
            x_val = get_img_feats(x[idx])
            #print(tens.shape)
            tensor = torch.tensor(x_val, dtype=torch.float32)
            tensor = tensor.flatten()
            y_val = label_oh_tf(y[idx], device, num_classes=11)
            #print(label)
            
            val_prediction = model.forward(tensor)
            #print(val_prediction.shape)
            v_loss = loss_fn(val_prediction, y_val)
            
            v_loss_list.append(v_loss)
            if val_prediction.argmax() == y_val.argmax():
                v_correct +=1
            
            val_acc = (v_correct / (len(val_ids)*(epoch+1)))*100
            v_loss_list.append(v_loss)
            v_label_list.append(y_val)
            
        print('Val Acc:   ',val_acc) 
        
            #print('val accuracy:     ', v_correct/len(val_ids))
            #val_avg_f1_score, val_acc = metrics(y_val, val_prediction)
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = deepcopy(model)
            
            save_dict['Current_Epoch'] += config['epochs']
            save_dict['model.state_dict'] = model.state_dict()
            save_dict['training_samples'] = len(train_ids)
            save_dict['validation_samples'] = len(val_ids)
            save_dict['t_loss_list'] = t_loss_list
            save_dict['t_predict_list'] = t_predict_list  
            save_dict['t_accuracy_list'] = t_accuracy_list  #
            save_dict['v_loss_list'] = v_loss_list
            save_dict['v_predict_list'] = v_predict_list  #
            save_dict['v_accuracy_list'] = v_accuracy_list  #
            save_dict['t_labels'] = t_label_list
            save_dict['v_labels'] = v_label_list
            title = save_dict['Run']
            with open(f'{save_location}{title}_acc{int(best_acc)}_epoch{total_epochs}', 'wb+') as f:
                pickle.dump(save_dict, f)
            
            
            print('improvment in metrics. model saved')
        
        if (epoch+1)%2==0:
            print('updating wandb')
            train_log(t_loss, v_loss, epoch)
            wandb.log({'train_accuracy_%': train_acc, 'epoch':epoch})
            wandb.log({'val_accuracy_%': val_acc, 'epoch':epoch})
            
    model= best_model
    labels = zip(t_label_list, v_label_list)
    losses = zip(t_loss_list, v_loss_list)
    predictions = zip(t_predict_list, v_predict_list)
    return model, losses, predictions, labels



def pipeline(config): 

    title = f"IDSW_on_RESNET101_Convs_training_MLP_test1"
    
    loss_list=[]
    #loss_fn = nn.CrossEntropyLoss()
    loss_fn = nn.MSELoss()
    with wandb.init(project=title, config=config):
        config = wandb.config
        model = Three_Lay_MLP().to(device)

        model, loss_list, predictions, labels = train_model(model, loss_fn,  train_ids, val_ids, config) #train_dl, val_dl
        
    return model, loss_list, predictions, labels

In [ ]:
model, loss_list, predictions, labels = pipeline(config) #7,168

  0%|          | 0/40 [00:00<?, ?it/s]

Epoch:    0
training...


<ipython-input-13-9c1779800324>:91: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(x_train, dtype=torch.float32)
/its/home/nn268/.local/lib/python3.8/site-packages/torch/nn/modules/container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


validation... 


<ipython-input-13-9c1779800324>:122: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(x_val, dtype=torch.float32)


...

improvment in metrics. model saved
updating wandb
Epoch:    32
training...
validation... 
Val Acc:    324.0


 82%|████████▎ | 33/40 [1:23:10<19:24, 166.39s/it]

improvment in metrics. model saved
Epoch:    33
training...
validation... 
Val Acc:    333.8181818181818


 85%|████████▌ | 34/40 [1:25:55<16:36, 166.05s/it]

improvment in metrics. model saved
updating wandb
Epoch:    34
training...


In [ ]:
print(loss_list)